In [42]:
import pandas as pd
import numpy as np

np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from sklearn.model_selection import train_test_split

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [78]:
# read cleaned data

df = pd.read_csv("new.csv")
df.head()

# df = df.drop(['Controller mistyrose', 'Humidity'], axis=1)
# train, test = train_test_split(df, test_size=0.05)
# data = df.to_numpy()

# df2 = df["Running"] + df["Controller mistyrose"] + df["Controller moccasin"]

# new = old[['A', 'C', 'D']].copy()
# df = df[["Running", "Controller mistyrose", "Controller moccasin"]].copy()

df.head()

,Running,Blue Switch On,Battery level,Humidity,Magnetic field,Controller beige,Controller bisque,Controller black,Controller blanchedalmond,Time since restart,...,Waste level right tank,Waste level left tank,Helium reservoir,alpha-radiation,Active motion supression,Blade density,Blade rotation,Controller mintcream,Controller mistyrose,Controller moccasin
0,1.0,1.0,0.472619,0.681050,0.884694,0.888819,0.949818,0.656389,0.788241,8.031743,...,0.921479,0.759088,0.742684,0.958868,0.546771,0.736409,0.471570,0.986777,0.600762,0.412784
1,1.0,1.0,0.688965,0.988019,0.716011,0.567877,0.463229,0.424654,0.931444,5.165232,...,0.562749,0.932735,0.684856,0.914809,0.972892,0.450422,0.471924,0.453124,0.976749,0.597788
2,1.0,1.0,0.788401,0.752765,0.801923,0.847497,0.743526,0.931443,0.768209,9.436875,...,0.881300,0.417064,0.468547,0.796907,0.986931,0.847634,0.728937,0.895689,0.970808,0.752744
3,0.0,1.0,0.739938,0.517953,0.431417,0.894524,0.752744,0.907213,0.871284,3.651928,...,0.926289,0.464410,0.958722,0.520464,0.999992,0.838839,0.506759,0.662043,0.659098,0.953559
4,1.0,1.0,0.525988,0.532473,0.920489,0.898508,0.707683,0.743687,0.726896,6.582867,...,0.434560,0.746746,0.466800,0.942878,0.781556,0.850591,0.592054,0.549517,0.574893,0.472128


In [85]:

# Creating Traning, Validation and Test Set

train, val, test = \
              np.split(df.sample(frac=1, random_state=38), 
                       [int(.9*len(df)), int(.95*len(df))])

train_data_features = train.copy()
train_data_labels = train_data_features.pop('Running')
train_data_features = np.array(train_data_features)
train_data_labels = np.array(train_data_labels)

test_data_features = test.copy()
test_data_labels = test_data_features.pop('Running')
test_data_features = np.array(test_data_features)
test_data_labels = np.array(test_data_labels)

val_data_features = val.copy()
val_data_labels = val_data_features.pop('Running')
val_data_features = np.array(val_data_features)
val_data_labels = np.array(val_data_labels)



In [86]:

# Callback for network analysis

def init_callbacks():

        return [

            # tf.keras.callbacks.TensorBoard(log_dir=self.config["logs"], profile_batch=0),
            tf.keras.callbacks.ReduceLROnPlateau(patience=20),
            # tf.keras.callbacks.EarlyStopping(patience=20, min_delta=0.01, verbose=1),
            tf.keras.callbacks.ModelCheckpoint(filepath='/home/cezary/Desktop/The Mysterious Machine-20210330/weights/test.h5',
                                               save_best_only=True,
                                               save_weights_only=True,
                                               verbose=1, monitor='val_loss'),
        ]

In [87]:

# normalization of the data
normalize = preprocessing.Normalization()
normalize.adapt(train_data_features)
normalize.adapt(val_data_features)

# small non linear network
model = tf.keras.Sequential([
  normalize,
  layers.Dense(64, activation='relu'), 
  layers.Dense(1)
])

model.compile(loss = tf.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(),
                      metrics=['accuracy'])

model.fit(train_data_features, train_data_labels, epochs=40, batch_size=16, 
          validation_data=(val_data_features, val_data_labels), callbacks=init_callbacks())



Epoch 1/40
 82/113 [====================>.........] - ETA: 0s - loss: 4.1244 - accuracy: 0.5473
Epoch 00001: val_loss improved from inf to 3.33432, saving model to /home/cezary/Desktop/The Mysterious Machine-20210330/weights/test.h5
113/113 [==============================] - 0s 2ms/step - loss: 3.9918 - accuracy: 0.5660 - val_loss: 3.3343 - val_accuracy: 0.6600
Epoch 2/40
 77/113 [===================>..........] - ETA: 0s - loss: 3.3127 - accuracy: 0.6615
Epoch 00002: val_loss improved from 3.33432 to 3.23732, saving model to /home/cezary/Desktop/The Mysterious Machine-20210330/weights/test.h5
113/113 [==============================] - 0s 2ms/step - loss: 3.4588 - accuracy: 0.6496 - val_loss: 3.2373 - val_accuracy: 0.6500
Epoch 3/40
 73/113 [==================>...........] - ETA: 0s - loss: 3.0415 - accuracy: 0.6807
Epoch 00003: val_loss improved from 3.23732 to 2.59655, saving model to /home/cezary/Desktop/The Mysterious Machine-20210330/weights/test.h5
113/113 [======================

 79/113 [===================>..........] - ETA: 0s - loss: 0.4214 - accuracy: 0.9533
Epoch 00027: val_loss did not improve from 1.24234
113/113 [==============================] - 0s 2ms/step - loss: 0.4801 - accuracy: 0.9499 - val_loss: 1.3416 - val_accuracy: 0.8400
Epoch 28/40
 72/113 [==================>...........] - ETA: 0s - loss: 0.4265 - accuracy: 0.9609
Epoch 00028: val_loss did not improve from 1.24234
113/113 [==============================] - 0s 1ms/step - loss: 0.4693 - accuracy: 0.9554 - val_loss: 1.3376 - val_accuracy: 0.8400
Epoch 29/40
 86/113 [=====================>........] - ETA: 0s - loss: 0.4400 - accuracy: 0.9637
Epoch 00029: val_loss did not improve from 1.24234
113/113 [==============================] - 0s 1ms/step - loss: 0.4621 - accuracy: 0.9604 - val_loss: 1.3369 - val_accuracy: 0.8500
Epoch 30/40
 83/113 [=====================>........] - ETA: 0s - loss: 0.4579 - accuracy: 0.9654
Epoch 00030: val_loss did not improve from 1.24234
113/113 [==================

In [88]:

# load weights and evaluate on test set

model.load_weights('/home/cezary/Desktop/The Mysterious Machine-20210330/weights/test.h5')

results = model.evaluate(test_data_features, test_data_labels, batch_size=128)

1/1 [==============================] - 0s 955us/step - loss: 0.9803 - accuracy: 0.8400


In [89]:

# some small experiments with classic ml algorithms

train_data_features = train.copy()
train_data_labels = train_data_features.pop('Running')

X_train = train_data_features
Y_train = train_data_labels

test_data_features = test.copy()
test_data_labels = test_data_features.pop('Running')

X_test = test_data_features
Y_test = test_data_labels

svc = SVC()
svc.fit(X_train, Y_train)
# Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_test, Y_test) * 100, 2)
print(acc_svc)

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
# Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_test, Y_test) * 100, 2)
print(acc_decision_tree)

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
# Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_test, Y_test) * 100, 2)
print(acc_knn)

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
# Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_test, Y_test) * 100, 2)
print(acc_gaussian)

72.0
91.0
64.0
48.0
